In [275]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import os
import numpy as np
import pandas as pd
import wandb
import plotly.express as px

# Pytorch implementation

In [276]:
use_wandb = False

In [277]:
config = {
    "learning_rate": 1e-3,
    "dataset": "Titanic",
    "epochs": 500,
    "batch_size": 16,
    "loss": "binary cross-entropy",
    "optimizer": "Adam"
}

In [278]:
if use_wandb:
    wandb.login()
    wandb.init(project="assignment-1", name="pytorch", reinit=True)
    wandb.config = config

In [279]:
# device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
device = 'cpu'
print(f"Using {device} device")

Using cpu device


## Loading the dataset

In [280]:
class TitanicDataset(Dataset):
    def __init__(self, data_dir):
        self.labels = pd.read_csv(os.path.join(data_dir, 'labels.csv')).to_numpy(dtype='float32')
        self.data = pd.read_csv(os.path.join(data_dir, 'data.csv')).to_numpy(dtype='float32')

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        data = torch.from_numpy(self.data[idx])
        label = torch.from_numpy(self.labels[idx])
        # data = self.data[idx]
        # label = self.labels[idx]

        return data, label

In [281]:
train_data = TitanicDataset(data_dir='data/train/')
val_data = TitanicDataset(data_dir='data/val/')
test_data = TitanicDataset(data_dir='data/test/')

In [282]:
features, label = train_data.__getitem__(idx=0)
print(features, label)

tensor([  1.0000,  49.0000,   1.0000,   1.0000, 110.8833]) tensor([0.])


In [283]:
train_dataloader = DataLoader(train_data, batch_size=config['batch_size'], shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=config['batch_size'], shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=config['batch_size'], shuffle=True)

In [284]:
for batch, (x, y) in enumerate(train_dataloader):
    print(batch, x.shape, y.shape)
    break

0 torch.Size([16, 5]) torch.Size([16, 1])


## Defining the model

In [285]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Sequential(
            nn.Linear(5, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        out = self.linear(x)
        return out

In [286]:
model = NeuralNetwork().to(device)
print(model)

if use_wandb: 
    wandb.config['model'] = model.__dict__['_modules']['linear']

NeuralNetwork(
  (linear): Sequential(
    (0): Linear(in_features=5, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


## Training

In [287]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])

In [288]:
# wandb.watch(model, loss_fn, log='all', log_freq=10)

In [289]:
def calculate_acc(y_pred, y) -> float:
    y_pred = y_pred>0.5

    # print(((y == y_pred).sum().item()))
    return ((y == y_pred).sum().item())/len(y)

In [290]:
loss_graph = []
acc_graph = []

def train(dataloader, model, loss_fn, optimizer, epoch):
    model.train()
    avg_loss, avg_acc = 0, 0
    curr = 0
    for x, y in dataloader:
        curr += len(y)
        x, y = x.to(device), y.to(device)

        # Get prediction and compute loss
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        avg_loss += loss.item()

        # Update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_acc += calculate_acc(y_pred, y)
        # print(f'loss: {loss.item():>5f} acc: {100*correct:>5f} | {curr}/{len(dataloader.dataset)}')

    avg_loss /= len(dataloader)
    avg_acc /= len(dataloader)
    
    loss_graph.append(avg_loss)
    acc_graph.append(avg_acc)
    
    if (epoch % 50 == 0):
        print(f"Epoch {epoch}")
        print(f'avg loss: {avg_loss:>5f} avg acc: {avg_acc:>5f}')
    
    if use_wandb:
        wandb.log({'loss': avg_loss})
        wandb.log({'acc': avg_acc})

In [291]:
val_loss_graph = []
val_acc_graph = []

def val(dataloader, model, loss_fn, epoch):    
    model.eval()
    avg_loss, avg_acc = 0, 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            avg_loss += loss_fn(y_pred, y).item()
            avg_acc += calculate_acc(y_pred, y)
    
    avg_loss /= len(dataloader)
    avg_acc /= len(dataloader)

    val_loss_graph.append(avg_loss)
    val_acc_graph.append(avg_acc)
    
    if (epoch % 50 == 0):
        print(f'avg val loss: {avg_loss:>5f} avg val acc: {avg_acc:>5f}')
        print('-------------------------------')
    
    if use_wandb:
        wandb.log({'val_loss': avg_loss})
        wandb.log({'val_acc': avg_acc})

In [292]:
epochs = config['epochs']
for epoch in range(epochs):
    # print(f"Epoch {epoch+1}")
    train(train_dataloader, model, loss_fn, optimizer, epoch)
    val(val_dataloader, model, loss_fn, epoch)
    # print('-------------------------------')

Epoch 0
avg loss: 0.748065 avg acc: 0.644231
avg val loss: 0.658687 avg val acc: 0.601389
-------------------------------
Epoch 50
avg loss: 0.565548 avg acc: 0.718483
avg val loss: 0.610917 avg val acc: 0.697222
-------------------------------
Epoch 100
avg loss: 0.541790 avg acc: 0.727564
avg val loss: 0.559436 avg val acc: 0.720833
-------------------------------
Epoch 150
avg loss: 0.537718 avg acc: 0.739850
avg val loss: 0.587714 avg val acc: 0.684722
-------------------------------
Epoch 200
avg loss: 0.518203 avg acc: 0.744124
avg val loss: 0.534724 avg val acc: 0.750000
-------------------------------
Epoch 250
avg loss: 0.513596 avg acc: 0.746795
avg val loss: 0.597510 avg val acc: 0.695833
-------------------------------
Epoch 300
avg loss: 0.498171 avg acc: 0.751603
avg val loss: 0.552953 avg val acc: 0.712500
-------------------------------
Epoch 350
avg loss: 0.490758 avg acc: 0.770833
avg val loss: 0.522155 avg val acc: 0.777778
-------------------------------
Epoch 400
a

In [293]:
if use_wandb:
    wandb.finish()

In [294]:
fig = px.line({'loss': loss_graph, 'val_loss': val_loss_graph})
fig.show()

In [295]:
fig = px.line({'acc': acc_graph, 'val_acc': val_acc_graph})
fig.show()